In [1]:
import os
import numpy as np
import pandas as pd
from datetime import date

from sklearn.model_selection import KFold, train_test_split, StratifiedKFold, cross_val_score, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import log_loss, roc_auc_score, auc, roc_curve
from sklearn.preprocessing import MinMaxScaler

DATA_ROOT = "./input/"


In [2]:
dfoff = pd.read_csv(os.path.join(DATA_ROOT,'train_offline.csv'))
dftest = pd.read_csv(os.path.join(DATA_ROOT,'test_offline.csv'))
dftest = dftest[~dftest.Coupon_id.isna()]
dftest.reset_index(drop=True, inplace=True)
print(dfoff.shape)
print(dftest.shape)
dfoff.head(20)

(1160742, 7)
(306313, 6)


,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date
0,1439408,2632,NaN,NaN,0.0,NaN,20160217.0
1,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN
2,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN
3,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN
4,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN
5,2223968,3381,9776.0,10:5,2.0,20160129.0,NaN
6,73611,2099,12034.0,100:10,NaN,20160207.0,NaN
7,163606,1569,5054.0,200:30,10.0,20160421.0,NaN
8,3273056,4833,7802.0,200:20,10.0,20160130.0,NaN
9,94107,3381,7610.0,200:20,2.0,20160412.0,NaN


In [3]:
## Creat target label 
"""
According to the definition, 
1) buy with coupon within (include) 15 days ==> 1
2) buy with coupon but out of 15 days ==> 0
3) buy without coupon ==> -1 (we don't care)
"""
def label(row):
    if np.isnan(row['Date_received']):
        return -1
    if not np.isnan(row['Date']):
        td = pd.to_datetime(row['Date'], format='%Y%m%d') -  pd.to_datetime(row['Date_received'], format='%Y%m%d')
        if td <= pd.Timedelta(15, 'D'):
            return 1
    return 0

dfoff["label"] = dfoff.apply(label, axis=1)
dfoff["label"].value_counts()



 0    710665
-1    413773
 1     36304
Name: label, dtype: int64

In [4]:
print(dfoff.shape)
dfoff=dfoff[dfoff['label']!=-1]
dfoff.shape

(1160742, 8)


(746969, 8)

In [5]:
# Generate features - weekday acquired coupon
def getWeekday(row):
    if (np.isnan(row)) or (row==-1):
        return row
    else:
        return pd.to_datetime(row, format = "%Y%m%d").dayofweek+1 # add one to make it from 0~6 -> 1~7

dfoff['weekday'] = dfoff['Date_received'].apply(getWeekday)
dftest['weekday'] = dftest['Date_received'].apply(getWeekday)

# weekday_type (weekend = 1)
# dfoff['weekday_type'] = dfoff['weekday'].astype('str').apply(lambda x : 1 if x in [6,7] else 0 ) # apply to trainset
# dftest['weekday_type'] = dftest['weekday'].astype('str').apply(lambda x : 1 if x in [6,7] else 0 ) # apply to testset
dfoff['weekday_type'] = dfoff['weekday'].astype('int').apply(lambda x : 1 if x in [6,7] else 0 ) # apply to trainset
dftest['weekday_type'] = dftest['weekday'].astype('int').apply(lambda x : 1 if x in [6,7] else 0 ) # apply to testset

In [6]:
weekdaycols = ['weekday_' + str(i) for i in range(1,8)]
print(weekdaycols)

tmpdf = pd.get_dummies(dfoff['weekday'].replace(-1, np.nan))
tmpdf.columns = weekdaycols
dfoff[weekdaycols] = tmpdf

tmpdf = pd.get_dummies(dftest['weekday'].replace(-1, np.nan))
tmpdf.columns = weekdaycols
dftest[weekdaycols] = tmpdf

['weekday_1', 'weekday_2', 'weekday_3', 'weekday_4', 'weekday_5', 'weekday_6', 'weekday_7']


In [7]:
import math
# Generate features - coupon discount and distance
def getDiscountType(row):
    if row == 'null':
        return 'null'
    elif ':' in row:
        return 1
    else:
        return 0

def convertRate(row):
#     print(row)
    """Convert discount to rate"""
    if ((row == 'null') or (row == 'nan')):
        return 1.0
    elif ':' in row:
        rows = row.split(':')
        return 1.0 - float(rows[1])/float(rows[0])
    else:
        return float(row)

def getDiscountMan(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[0])
    else:
        return 0

def getDiscountJian(row):
    if ':' in row:
        rows = row.split(':')
        return int(rows[1])
    else:
        return 0

def processData(df):
    
    # convert discunt_rate
    df['discount_rate'] = df['Discount_rate'].astype('str').apply(convertRate)
    df['discount_man'] = df['Discount_rate'].astype('str').apply(getDiscountMan)
    df['discount_jian'] = df['Discount_rate'].astype('str').apply(getDiscountJian)
    df['discount_type'] = df['Discount_rate'].astype('str').apply(getDiscountType)
    
    # convert distance
    df.loc[df.Distance.isna(), "Distance"] = 99
    return df

dfoff = processData(dfoff)
dftest = processData(dftest)

In [8]:
# Add Shopping_count
# offline
count_dfoff = dfoff['User_id'].value_counts().reset_index()
count_dfoff.columns=['User_id', 'Shopping_count']
dfoff = pd.merge(dfoff, count_dfoff, on=['User_id'], how='left')

# test data
count_dftest = dftest['User_id'].value_counts().reset_index()
count_dftest.columns=['User_id', 'Shopping_count']
dftest = pd.merge(dftest, count_dftest, on=['User_id'], how='left')


In [9]:
dfoff.head()

,User_id,Merchant_id,Coupon_id,Discount_rate,Distance,Date_received,Date,label,weekday,weekday_type,...,weekday_3,weekday_4,weekday_5,weekday_6,weekday_7,discount_rate,discount_man,discount_jian,discount_type,Shopping_count
0,1439408,2632,8591.0,20:1,0.0,20160217.0,NaN,0,3,0,...,1,0,0,0,0,0.95,20,1,1,2
1,1439408,2632,1078.0,20:1,0.0,20160319.0,NaN,0,6,1,...,0,0,0,1,0,0.95,20,1,1,2
2,1832624,3381,7610.0,200:20,0.0,20160429.0,NaN,0,5,0,...,0,0,1,0,0,0.90,200,20,1,1
3,2029232,3381,11951.0,200:20,1.0,20160129.0,NaN,0,5,0,...,0,0,1,0,0,0.90,200,20,1,1
4,2223968,3381,9776.0,10:5,2.0,20160129.0,NaN,0,5,0,...,0,0,1,0,0,0.50,10,5,1,1


In [10]:
train_Y = dfoff["label"]
targetPars = ['discount_rate',
                    'discount_type',
                    'discount_man', 
                    'discount_jian',
                    'Distance', 
                    'weekday', 
                    'weekday_type',
                     'Shopping_count'] + weekdaycols
lengthOfFeatures = len(targetPars)
train_X = dfoff[targetPars]

# Y_test = dftest["label"]
test_X = dftest[targetPars]

# Y_test


In [11]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
lr = LogisticRegression(tol=0.001, penalty='l2', fit_intercept=True, C=1.0)


lr.fit(train_X, train_Y)



C:\Users\ychu\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.001, verbose=0, warm_start=False)

In [12]:
ls_pred = lr.predict_proba(test_X)[:,1]
targetset= dftest.copy()
targetset = targetset[~targetset.Coupon_id.isna()]
targetset.reset_index(drop=True, inplace=True)

test1 = test_X.copy()
test1['pred_prob'] = ls_pred

output = pd.concat((targetset[["User_id", "Coupon_id", "Date_received"]], test1["pred_prob"]), axis=1)
print(output.shape)
output.loc[:, "User_id"] = output["User_id"].apply(lambda x:str(int(x)))
output.loc[:, "Coupon_id"] = output["Coupon_id"].apply(lambda x:str(int(x)))
output.loc[:, "Date_received"] = output["Date_received"].apply(lambda x:str(int(x)))
output["uid"] = output[["User_id", "Coupon_id", "Date_received"]].apply(lambda x: '_'.join(x.values), axis=1)
output.reset_index(drop=True, inplace=True)

### NOTE: YOUR SUBMITION FILE SHOULD HAVE COLUMN NAME: uid, label
out = output.groupby("uid", as_index=False).mean()
out = out[["uid", "pred_prob"]]
out.columns = ["uid", "label"]
out.to_csv("lr.csv", header=["uid", "label"], index=False) # submission format
out.head()

(306313, 4)


,uid,label
0,1000020_2705_20160519,0.103688
1,1000020_8192_20160513,0.080328
2,1000065_1455_20160527,0.063972
3,1000085_8067_20160513,0.062665
4,1000086_2418_20160613,0.049892


In [13]:
rf = RandomForestClassifier(n_estimators=10, min_samples_split=2, min_samples_leaf=1, 
                            max_features='sqrt', max_depth=6, bootstrap=True)
rf.fit(train_X, train_Y)

feats = pd.Series(data=rf.feature_importances_, index=train_X.columns)
feats = feats.sort_values(ascending=False)
feats

Distance          0.292728
discount_man      0.282208
discount_jian     0.162307
Shopping_count    0.156065
discount_rate     0.068993
discount_type     0.011621
weekday           0.011445
weekday_2         0.003826
weekday_7         0.003742
weekday_1         0.002136
weekday_3         0.001347
weekday_type      0.001082
weekday_4         0.000934
weekday_6         0.000855
weekday_5         0.000713
dtype: float64

In [14]:
rf_pred = rf.predict_proba(test_X)[:,1]
targetset= dftest.copy()
targetset = targetset[~targetset.Coupon_id.isna()]
targetset.reset_index(drop=True, inplace=True)

test1 = test_X.copy()
test1['pred_prob'] = rf_pred

output = pd.concat((targetset[["User_id", "Coupon_id", "Date_received"]], test1["pred_prob"]), axis=1)
print(output.shape)
output.loc[:, "User_id"] = output["User_id"].apply(lambda x:str(int(x)))
output.loc[:, "Coupon_id"] = output["Coupon_id"].apply(lambda x:str(int(x)))
output.loc[:, "Date_received"] = output["Date_received"].apply(lambda x:str(int(x)))
output["uid"] = output[["User_id", "Coupon_id", "Date_received"]].apply(lambda x: '_'.join(x.values), axis=1)
output.reset_index(drop=True, inplace=True)

### NOTE: YOUR SUBMITION FILE SHOULD HAVE COLUMN NAME: uid, label
out = output.groupby("uid", as_index=False).mean()
out = out[["uid", "pred_prob"]]
out.columns = ["uid", "label"]
out.to_csv("rf.csv", header=["uid", "label"], index=False) # submission format
out.head()

(306313, 4)


,uid,label
0,1000020_2705_20160519,0.198780
1,1000020_8192_20160513,0.174166
2,1000065_1455_20160527,0.119221
3,1000085_8067_20160513,0.127095
4,1000086_2418_20160613,0.110523


In [15]:
gdbt = GradientBoostingClassifier(subsample=0.75, n_estimators=200, max_features=lengthOfFeatures,
                                  max_depth=5, learning_rate=0.03, tol=0.1)
gdbt.fit(train_X, train_Y)

feats = pd.Series(data=gdbt.feature_importances_, index=train_X.columns)
feats = feats.sort_values(ascending=False)
feats


Distance          0.323215
discount_man      0.245340
Shopping_count    0.235848
discount_rate     0.107382
discount_jian     0.043720
weekday           0.021290
weekday_5         0.010253
weekday_7         0.004643
discount_type     0.002692
weekday_3         0.001266
weekday_1         0.001185
weekday_2         0.001098
weekday_6         0.000930
weekday_4         0.000844
weekday_type      0.000294
dtype: float64

In [16]:
gdbt_pred = gdbt.predict_proba(test_X)[:,1]
targetset= dftest.copy()
targetset = targetset[~targetset.Coupon_id.isna()]
targetset.reset_index(drop=True, inplace=True)

test1 = test_X.copy()
test1['pred_prob'] = gdbt_pred

output = pd.concat((targetset[["User_id", "Coupon_id", "Date_received"]], test1["pred_prob"]), axis=1)
print(output.shape)
output.loc[:, "User_id"] = output["User_id"].apply(lambda x:str(int(x)))
output.loc[:, "Coupon_id"] = output["Coupon_id"].apply(lambda x:str(int(x)))
output.loc[:, "Date_received"] = output["Date_received"].apply(lambda x:str(int(x)))
output["uid"] = output[["User_id", "Coupon_id", "Date_received"]].apply(lambda x: '_'.join(x.values), axis=1)
output.reset_index(drop=True, inplace=True)

### NOTE: YOUR SUBMITION FILE SHOULD HAVE COLUMN NAME: uid, label
out = output.groupby("uid", as_index=False).mean()
out = out[["uid", "pred_prob"]]
out.columns = ["uid", "label"]
out.to_csv("gdbt.csv", header=["uid", "label"], index=False) # submission format
out.head()

(306313, 4)


,uid,label
0,1000020_2705_20160519,0.219807
1,1000020_8192_20160513,0.190627
2,1000065_1455_20160527,0.068220
3,1000085_8067_20160513,0.091623
4,1000086_2418_20160613,0.105604
